# PySpark ML

In [2]:
%fs ls dbfs:/FileStore/tables/

path,name,size
dbfs:/FileStore/tables/01_pyspark_ml.ipynb,01_pyspark_ml.ipynb,19104
dbfs:/FileStore/tables/births.csv,births.csv,1908068
dbfs:/FileStore/tables/births_transformed.csv,births_transformed.csv,1908068


## Predict chances of infant survival with ML

### Load the data

In [5]:
from pyspark.sql.types import *

labels = [
    ('INFANT_ALIVE_AT_REPORT', IntegerType()),
    ('BIRTH_PLACE', IntegerType()),
    ('MOTHER_AGE_YEARS', IntegerType()),
    ('FATHER_COMBINED_AGE', IntegerType()),
    ('CIG_BEFORE', IntegerType()),
    ('CIG_1_TRI', IntegerType()),
    ('CIG_2_TRI', IntegerType()),
    ('CIG_3_TRI', IntegerType()),
    ('MOTHER_HEIGHT_IN', IntegerType()),
    ('MOTHER_PRE_WEIGHT', IntegerType()),
    ('MOTHER_DELIVERY_WEIGHT', IntegerType()),
    ('MOTHER_WEIGHT_GAIN', IntegerType()),
    ('DIABETES_PRE', IntegerType()),
    ('DIABETES_GEST', IntegerType()),
    ('HYP_TENS_PRE', IntegerType()),
    ('HYP_TENS_GEST', IntegerType()),
    ('PREV_BIRTH_PRETERM', IntegerType())
]

schema = StructType([
    StructField(e[0], e[1], False) for e in labels
])

births = spark.read.csv('dbfs:/FileStore/tables/births.csv', 
                        header=True, 
                        schema=schema)

In [6]:
births.show()

+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
INFANT_ALIVE_AT_REPORT|BIRTH_PLACE|MOTHER_AGE_YEARS|FATHER_COMBINED_AGE|CIG_BEFORE|CIG_1_TRI|CIG_2_TRI|CIG_3_TRI|MOTHER_HEIGHT_IN|MOTHER_PRE_WEIGHT|MOTHER_DELIVERY_WEIGHT|MOTHER_WEIGHT_GAIN|DIABETES_PRE|DIABETES_GEST|HYP_TENS_PRE|HYP_TENS_GEST|PREV_BIRTH_PRETERM|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
 0| 1| 29| 99| 0| 0| 0| 0| 99| 999| 999| 99| 0| 0| 0| 0| 0|
 0| 1| 22| 29| 0| 0| 0| 0| 65| 180| 198| 18| 0| 0| 0| 0| 0|
 0| 1| 38| 40| 0| 0| 0| 0| 63| 155| 167| 12| 0| 0| 0| 0| 0|
 0| 1| 39| 42| 0| 0| 0| 0| 60| 128| 152| 24| 0| 0| 0| 0| 1|
 0| 1| 18| 99| 6| 4| 2| 2| 61| 110| 130| 20| 0| 0| 0| 0| 0|
 0| 1| 32| 37| 0| 0| 0| 0| 66| 150| 162| 12| 0| 0| 0| 0| 0|
 0| 1| 22| 25| 0| 0| 0| 0| 68| 155| 191| 36| 0| 0| 0| 0| 0|
 0| 1| 25| 26| 0| 0| 0| 0| 64| 136| 169| 33| 0| 0| 0| 0| 0|
 0| 1| 26| 32| 0| 0| 0| 0| 64| 140| 147| 7| 0| 0| 0| 0| 0|
 0| 1| 39| 66| 0| 0| 0| 0| 65| 140| 150| 10| 0| 0| 0| 0| 0|
 0| 1| 25| 22| 0| 0| 0| 0| 62| 145| 152| 7| 0| 0| 0| 0| 0|
 0| 1| 33| 99| 0| 0| 0| 0| 65| 145| 145| 0| 0| 0| 0| 0| 0|
 0| 1| 25| 99| 0| 0| 0| 0| 64| 140| 165| 25| 0| 0| 0| 0| 0|
 0| 1| 29| 99| 0| 0| 0| 0| 60| 115| 120| 5| 0| 0| 0| 0| 0|
 0| 1| 28| 29| 0| 0| 0| 0| 66| 320| 318| 0| 0| 0| 1| 0| 1|
 0| 1| 23| 28| 0| 0| 0| 0| 64| 120| 141| 21| 0| 0| 0| 0| 0|
 0| 1| 31| 41| 0| 0| 0| 0| 59| 106| 142| 36| 0| 0| 0| 0| 1|
 0| 1| 27| 99| 0| 0| 0| 0| 66| 213| 200| 0| 0| 0| 0| 0| 0|
 0| 1| 28| 27| 0| 0| 0| 0| 66| 165| 224| 59| 0| 0| 0| 0| 1|
 0| 1| 34| 31| 0| 0| 0| 0| 70| 130| 134| 4| 0| 0| 0| 0| 0|
+----------------------+-----------+----------------+-------------------+----------+---------+---------+---------+----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+
only showing top 20 rows

In [7]:
births.groupby('BIRTH_PLACE').count().show()

+-----------+-----+
BIRTH_PLACE|count|
+-----------+-----+
 1|44558|
 6| 11|
 3| 224|
 5| 74|
 9| 8|
 4| 327|
 7| 91|
 2| 136|
+-----------+-----+

### Create transformers
- Having done this, we can now create our first `Transformer`.

In [9]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler

encoder = OneHotEncoder(
    inputCol='BIRTH_PLACE', 
    outputCol='BIRTH_PLACE_VEC')

Let's now create a single column with all the features collated together.

In [11]:
labels[0:]

Out[ 5 ]: 
[('INFANT_ALIVE_AT_REPORT', IntegerType),
 ('BIRTH_PLACE', IntegerType),
 ('MOTHER_AGE_YEARS', IntegerType),
 ('FATHER_COMBINED_AGE', IntegerType),
 ('CIG_BEFORE', IntegerType),
 ('CIG_1_TRI', IntegerType),
 ('CIG_2_TRI', IntegerType),
 ('CIG_3_TRI', IntegerType),
 ('MOTHER_HEIGHT_IN', IntegerType),
 ('MOTHER_PRE_WEIGHT', IntegerType),
 ('MOTHER_DELIVERY_WEIGHT', IntegerType),
 ('MOTHER_WEIGHT_GAIN', IntegerType),
 ('DIABETES_PRE', IntegerType),
 ('DIABETES_GEST', IntegerType),
 ('HYP_TENS_PRE', IntegerType),
 ('HYP_TENS_GEST', IntegerType),
 ('PREV_BIRTH_PRETERM', IntegerType)]

In [12]:
featuresCreator = VectorAssembler(
    inputCols=[
        col[0] 
        for col 
        in labels[2:]] + \
    [encoder.getOutputCol()], 
    outputCol='features'
)

In [13]:
[col[0] for col in labels[2:]] + [encoder.getOutputCol()]

Out[ 7 ]: 
['MOTHER_AGE_YEARS',
 'FATHER_COMBINED_AGE',
 'CIG_BEFORE',
 'CIG_1_TRI',
 'CIG_2_TRI',
 'CIG_3_TRI',
 'MOTHER_HEIGHT_IN',
 'MOTHER_PRE_WEIGHT',
 'MOTHER_DELIVERY_WEIGHT',
 'MOTHER_WEIGHT_GAIN',
 'DIABETES_PRE',
 'DIABETES_GEST',
 'HYP_TENS_PRE',
 'HYP_TENS_GEST',
 'PREV_BIRTH_PRETERM',
 'BIRTH_PLACE_VEC']

### Create an estimator
- Which machine learning model might be a good start? Why?

In [15]:
from pyspark.ml.classification import LogisticRegression

Once loaded, let's create the model.

In [17]:
logistic = LogisticRegression(
    maxIter=10, 
    regParam=0.01, 
    labelCol='INFANT_ALIVE_AT_REPORT')

### Create a pipeline

All that is left now is to creat a `Pipeline` and fit the model. First, let's load the `Pipeline` from the package.

In [20]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[
        encoder, 
        featuresCreator, 
        logistic
    ])

### Fit the model

Conventiently, `DataFrame` API has the `.randomSplit(...)` method.

In [23]:
births_train, births_test = births.randomSplit([0.7, 0.3], seed=1)

Now run our `pipeline` and estimate our model.

In [25]:
model = pipeline.fit(births_train)
test_model = model.transform(births_test)

Here's what the `test_model` looks like.

In [27]:
test_model.printSchema()

root
-- INFANT_ALIVE_AT_REPORT: integer (nullable = true)
-- BIRTH_PLACE: integer (nullable = true)
-- MOTHER_AGE_YEARS: integer (nullable = true)
-- FATHER_COMBINED_AGE: integer (nullable = true)
-- CIG_BEFORE: integer (nullable = true)
-- CIG_1_TRI: integer (nullable = true)
-- CIG_2_TRI: integer (nullable = true)
-- CIG_3_TRI: integer (nullable = true)
-- MOTHER_HEIGHT_IN: integer (nullable = true)
-- MOTHER_PRE_WEIGHT: integer (nullable = true)
-- MOTHER_DELIVERY_WEIGHT: integer (nullable = true)
-- MOTHER_WEIGHT_GAIN: integer (nullable = true)
-- DIABETES_PRE: integer (nullable = true)
-- DIABETES_GEST: integer (nullable = true)
-- HYP_TENS_PRE: integer (nullable = true)
-- HYP_TENS_GEST: integer (nullable = true)
-- PREV_BIRTH_PRETERM: integer (nullable = true)
-- BIRTH_PLACE_VEC: vector (nullable = true)
-- features: vector (nullable = true)
-- rawPrediction: vector (nullable = true)
-- probability: vector (nullable = true)
-- prediction: double (nullable = false)

Let's look only at the columns prediction, INFANT_ALIVE_AT_REPORT and features

In [29]:
test_model.select('prediction','INFANT_ALIVE_AT_REPORT','features').show(10)

+----------+----------------------+--------------------+
prediction|INFANT_ALIVE_AT_REPORT| features|
+----------+----------------------+--------------------+
 0.0| 0|(24,[0,1,6,7,8,9,...|
 1.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,6,7,8,16...|
 0.0| 0|(24,[0,1,6,7,8,16...|
 0.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,6,7,8,9,...|
 0.0| 0|(24,[0,1,2,3,4,5,...|
+----------+----------------------+--------------------+
only showing top 10 rows

### Model performance

How well did the model do?

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    rawPredictionCol='probability', 
    labelCol='INFANT_ALIVE_AT_REPORT')

print(evaluator.evaluate(test_model, 
     {evaluator.metricName: 'areaUnderROC'}))

0.730999898192873

THE END!

Other things you can do :
  - Use GridSearch to see what hyperparameters are the best for Logistic Regression
  - Try out another machine learning model such as Random Forest